In [15]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
from zipfile import ZipFile
z = ZipFile("/gdrive/My Drive/additional/train.zip")
z.extractall("train")

In [17]:
import glob
import pandas as pd
df1 = pd.DataFrame()
paths = glob.glob("train/train/cat.*")
df1["path"] = paths
df1["target"] = 0
df2 = pd.DataFrame()
paths = glob.glob("train/train/dog.*")
df2["path"] = paths
df2["target"] = 1
df = pd.concat([df1, df2], axis=0, ignore_index=True)
df

,path,target
0,train/train/cat.1593.jpg,0
1,train/train/cat.3661.jpg,0
2,train/train/cat.3246.jpg,0
3,train/train/cat.5287.jpg,0
4,train/train/cat.8281.jpg,0
5,train/train/cat.12309.jpg,0
6,train/train/cat.4506.jpg,0
7,train/train/cat.1483.jpg,0
8,train/train/cat.12403.jpg,0
9,train/train/cat.1167.jpg,0


In [25]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.layers import BatchNormalization
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
for l in vgg.layers:
    l.trainable = False
x = BatchNormalization()(vgg.output)
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

W0701 08:51:15.806059 140104115427200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

In [0]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
from sklearn.model_selection import train_test_split
import numpy as np
x_np = np.array(df["path"])
y_np = np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x_np, y_np, test_size=0.2)

In [0]:
from keras.utils.np_utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [22]:
# DEMO: 怎麼處理圖片 
# PIL: Pillow
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
import random
c = random.randint(0, x_train.shape[0] - 1)
img = np.array(load_img(x_train[c], target_size=(224, 224)))
preprocess_input(img)

array([[[-38.939003  , -57.779     , -53.68      ],
        [-33.939003  , -49.779     , -41.68      ],
        [-26.939003  , -37.779     , -26.68      ],
        ...,
        [-20.939003  , -27.779     , -15.68      ],
        [-24.939003  , -31.779     , -19.68      ],
        [-19.939003  , -27.779     , -13.68      ]],

       [[-15.939003  , -34.779     , -30.68      ],
        [-18.939003  , -34.779     , -26.68      ],
        [-22.939003  , -33.779     , -22.68      ],
        ...,
        [-25.939003  , -32.779     , -20.68      ],
        [-26.939003  , -33.779     , -21.68      ],
        [-21.939003  , -28.779     , -16.68      ]],

       [[-13.939003  , -30.779     , -26.68      ],
        [-20.939003  , -33.779     , -26.68      ],
        [-28.939003  , -39.779     , -28.68      ],
        ...,
        [-22.939003  , -28.779     , -18.68      ],
        [-18.939003  , -24.779     , -14.68      ],
        [-18.939003  , -25.779     , -13.68      ]],

       ...,

      

In [28]:
batch = 20
for i in range(5):
    idx = np.random.randint(0, x_train.shape[0], batch)
    ps = x_train[idx]
    img_list = []
    for fp in ps:
        img = load_img(fp, target_size=(224, 224))
        img_np = np.array(img)
        img_np_pre = preprocess_input(img_np)
        img_list.append(img_np_pre)
    img_list_np = np.array(img_list)
    targets = y_train_cat[idx]
    train_result = cnn.train_on_batch(img_list_np, targets)
    print("Train:", i, ":", train_result)

    test_idx = np.random.randint(0, x_test.shape[0], 20)
    test_ps = x_test[test_idx]
    test_img_list = []
    for fp in test_ps:
        test_img = load_img(fp, target_size=(224, 224))
        test_img_np = np.array(test_img)
        test_img_np_pre = preprocess_input(test_img_np)
        test_img_list.append(test_img_np_pre)
    test_img_list_np = np.array(test_img_list)
    test_targets = y_test_cat[test_idx]
    test_result = cnn.test_on_batch(test_img_list_np, test_targets)
    print("Test", i, ":", test_result)

Train: 0 : [1.7264175, 0.8]
Test 0 : [3.6087976, 0.7]
Train: 1 : [6.0533442, 0.5]
Test 1 : [1.5585325, 0.85]
Train: 2 : [0.9255661, 0.85]
Test 2 : [1.1213465, 0.85]
Train: 3 : [0.67838174, 0.95]
Test 3 : [0.80591804, 0.95]
Train: 4 : [0.61315906, 0.85]
Test 4 : [0.44439372, 0.9]


In [29]:
test_idx = np.random.randint(0, x_test.shape[0], 200)
test_ps = x_test[test_idx]
test_img_list = []
for fp in test_ps:
    test_img = load_img(fp, target_size=(224, 224))
    test_img_np = np.array(test_img)
    test_img_np_pre = preprocess_input(test_img_np)
    test_img_list.append(test_img_np_pre)
test_img_list_np = np.array(test_img_list)
test_targets = y_test_cat[test_idx]
cnn.evaluate(test_img_list_np, test_targets)

200/200 [==============================] - 102s 512ms/step


[0.8693663263320923, 0.895]